# Generic Functions

In [1]:
def split_path_and_fileName(strPathName):
    """
    Split the path and the name of the file of the given string 
    :INPUT:
        strPathName: str with the path and name of a file. ex: /data/pointcloud.txt
    :OUTPUT:
        list, ["/data/", "pointcloud.txt"]
    """
    for idx in range(len(strPathName)-1, 0, -1):
        if(strPathName[idx]=='/'):
            return strPathName[0:idx], strPathName[idx+1:]
    return None

# Predictions

## RandLA-NET

In [2]:
import sys 
from randlanet.main_apple_tree import *

/mnt/76db166d-2c59-4b55-9a91-19935005e2ef/miniconda3/envs/fh/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/mnt/76db166d-2c59-4b55-9a91-19935005e2ef/miniconda3/envs/fh/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/mnt/76db166d-2c59-4b55-9a91-19935005e2ef/miniconda3/envs/fh/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.d

In [3]:
path2data_rnet = "data/example2notebook_realdata/dataToRDNET/"  # Data to predict
path2model_rnet= "models/randlanet/continue_training_jitter08_rw0_8realTrees/snapshots/snap-8501" # Trained model 
                                                                                                    # to load
# The predicted labels are at the moment in the test folder

In [4]:
# Arguments for the model
param = {"gpu":0, # -1 no GPU
         "mode":"test",  # to Predict
         "model_path":path2model_rnet, 
         "path2data":path2data_rnet, 
         "path2output": "./", # This arg only works to save the training 
         "protocol":"field_only_xyz", 
         "restoreTrain":None}  
# NOTE: Ensure that the subsampling points in the training are the same for the prediction[test]

In [ ]:
# Methods to execute 
if(param["protocol"] == "synthetic"):
    train_synthetic_HiHiRes(param["path2data"], param["path2output"], parameters=param)
elif(param["protocol"] == "field_only_xyz"):
    train_field_only_xyz(param["path2data"], param["path2output"], parameters=param)
elif(param["protocol"] == "field"):
    train_field(param["path2data"], param["path2output"], parameters=param)
else:
    print("-> Error: Unknow options, please execute the following command and verify the defined args. \n$$>python main_apple_tree.py -h")

./
Train files :

Validation files :

Test files :


Preparing reprojection indices for validation and test
finished
Initiating input pipelines
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, use
    tf.py_function, which takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use keras.layers.batch_normalization instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the l

/mnt/76db166d-2c59-4b55-9a91-19935005e2ef/miniconda3/envs/fh/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [ ]:
# Merge labels 
# NOTE: RandLA-NET write the probabilities and the labels of each point cloud on different files, 
# To visualize the classification the predicted classes and the point cloud are going to be merged 
from randlanet.utils.merge_label_apple import *

In [ ]:
path2prediction = "test/Log_2021-02-01_06-02-17/predictions/" # The name of the folder always is 
                                                              # going to change with the date
path2data = os.path.join(param["path2data"],"test/")
output = os.path.join(path2data, "merged/")

merge_pointCloudAndLabels(path2data, "./test/", output)

## Random Forest 

In [ ]:
from machine_learning.predict import predict 
from machine_learning.RFClassifier import RFClassifier

In [ ]:
model = RFClassifier
model_weights = "data/example2notebook_realdata/model_rf.sav"
path2data = "data/example2notebook_realdata/dataToRDF/test/"
path2output = os.path.join(path2data, "prediction/")

In [ ]:
predict(model, model_weights, path2data, path2output)

# Clustering 

In [ ]:
import os 
import glob
import numpy as np 
import sklearn.cluster
from post_processing.algorithm import clustering 

In [ ]:
files_annApples = glob.glob(os.path.join("/media/juan/jprb/PhD_2020_2023/Data/data2fruithunter/annotated_apples_noRadiometric/","*.txt"))
path2wrt = "/media/juan/jprb/PhD_2020_2023/Data/data2fruithunter/annotated_clusters/"
eps, minSamples = 0.1, 20 # 0.4, 20 funciona pero consume mucha memoria 

print("Found annotated files: %i" %(len(files_annApples)))

for idx, file2clustering in enumerate(files_annApples, start=1):
    _, actualFileName = split_path_and_fileName(file2clustering)
    print("-> Loading[%i/%i]: %s" %(len(files_annApples), idx, actualFileName))
    pointCloud2cluster = np.loadtxt(file2clustering)
    cluster = clustering(pointCloud2cluster, minSamples, eps)
    print(" -> The file will be written in: %s" %path2wrt)
    np.savetxt(path2wrt+actualFileName, cluster)
    
    
    